In [ ]:
#импорт библиотек
import zipfile
import numpy as np
import pandas as pd
from IPython.display import display
from datetime import datetime
from dateutil.parser import parse
#!pip install category_encoders
import category_encoders as ce
from sklearn.linear_model import LinearRegression


#импорт библиотек



from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 

In [ ]:
zf = zipfile.ZipFile('../input/sberbank-russian-housing-market/train.csv.zip') 
df = pd.read_csv(zf.open('train.csv'), index_col = 'id')

In [ ]:
df.describe()

In [ ]:
df.timestamp = df.timestamp.apply(lambda x: parse(x))   # спарсили дату
df.reset_index(drop = True, inplace = True)
df.set_index(['timestamp'], inplace = True)         # установили дату как индекс

In [ ]:
df_obj = df[df.dtypes[df.dtypes == object].index]   #работа с object
encoder = ce.OneHotEncoder(cols = df_obj.columns, use_cat_names = True)
encoder.fit(df)
df = encoder.transform(df)        #все object-ы даммировали , положили сюда

In [ ]:
# теперь разберемся с пропусками
df_nan = df.isnull().sum().reset_index()
df_nan.columns = ['col_name', 'quant']
column_nan = df_nan[df_nan.quant > 0].reset_index(drop = True)
column_nan.quant = column_nan.quant.apply(lambda x: round((x*100)/30471, 1))
print("Количество столбцов с пропущенными значениями: {}".format(column_nan.shape[0]))
print("Максимальный процент пропущенных данных по столбцам: {}%".format(column_nan.quant.max()))
print("Минимальный процент пропущенных данных по столбцам: {}%".format(column_nan.quant.min()))
print("Медианное значение пропущенных данных по столбцам: {}%".format(round(column_nan.quant.median(),1)))

In [ ]:
# там, где мало пропусков заменили на среднее
df.floor = df[['floor']].fillna(df[['floor']].mean())
df.metro_min_walk = df[['metro_min_walk']].fillna(df[['metro_min_walk']].mean())
df.metro_km_walk = df[['metro_km_walk']].fillna(df[['metro_km_walk']].mean())
df.railroad_station_walk_km = df[['railroad_station_walk_km']].fillna(df[['railroad_station_walk_km']].mean())
df.railroad_station_walk_min = df[['railroad_station_walk_min']].fillna(df[['railroad_station_walk_min']].mean())
df.prom_part_5000 = df[['prom_part_5000']].fillna(df[['prom_part_5000']].mean())
df.cafe_avg_price_5000 = df[['cafe_avg_price_5000']].fillna(df[['cafe_avg_price_5000']].mean())
df.cafe_sum_5000_max_price_avg = df[['cafe_sum_5000_max_price_avg']].fillna(df[['cafe_sum_5000_max_price_avg']].mean())
df.cafe_sum_5000_min_price_avg = df[['cafe_sum_5000_min_price_avg']].fillna(df[['cafe_sum_5000_min_price_avg']].mean())
df.ID_railroad_station_walk = df[['ID_railroad_station_walk']].fillna(df[['ID_railroad_station_walk']].mean())

In [ ]:
def predict_value(y, x):
    x.append(y)
    df_predict = df[x]
    df_replacena_train=df_predict.loc[df[y].notna()]
    df_linear=df_replacena_train.drop(columns = [y],axis=1)
    df_linear_y=df_replacena_train[y].tolist()
    reg=LinearRegression()
    reg.fit(df_linear,df_linear_y)

    df_na=df_predict.loc[df_predict[y].isna()]
    df_linear_pred = df_na.drop(columns = [y])

    list_pred=reg.predict(df_linear_pred )
    list_pred_rounded=[]
    for i in list_pred:
        list_pred_rounded.append(i)

    
    df_na[y]=list_pred_rounded

    df_predict =pd.concat([df_na,df_replacena_train])
    df[y] = df_predict[y].values

In [ ]:
y = 'life_sq'
x = ['full_sq']
predict_value(y,x)

y = 'num_room'
x = ['full_sq']
predict_value(y,x)


y = 'kitch_sq'
x = ['full_sq']
predict_value(y,x)

In [ ]:
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
y = 'cafe_sum_500_min_price_avg'
predict_value(y,x)

y = 'cafe_sum_1500_max_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_avg_price_1500'
x = [ 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_2000_min_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_2000_max_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_3000_min_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_3000_max_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_500_max_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_avg_price_500'
x = [ 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_1000_min_price_avg'
x = [ 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_1000_max_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_avg_price_1000'
x = [ 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_sum_1500_min_price_avg'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)


y = 'cafe_avg_price_2000'
x = ['cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

y = 'cafe_avg_price_3000'
x = [ 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500' , 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high' ]
predict_value(y,x)

In [ ]:
df[df.floor > df.max_floor].max_floor = df[df.floor > df.max_floor].floor #заменила max_floor на floor
df.max_floor = df[['max_floor']].fillna(df[['max_floor']].mean())

In [ ]:
replace_state = df[df.state > 4]['state'].unique()
df['state'].replace(replace_state.tolist(),df[df.state < 5][['state']].mode().values[0][0], inplace = True)

In [ ]:
df.state = df[['state']].fillna(df[df.state < 5][['state']].mode().values[0][0])

In [ ]:
replace_year = df[df.build_year < 1691.0]['build_year'].unique()
df['build_year'].replace(replace_year.tolist(),df[df.build_year > 1691.0][['build_year']].mode().values[0][0], inplace = True)
replace_year = df[df.build_year > 2018.0]['build_year'].unique()
df['build_year'].replace(replace_year.tolist(),df[df.build_year > 1691.0][['build_year']].mode().values[0][0], inplace = True)

In [ ]:
for i in column_nan.col_name.values:
    df[i] = df[i].fillna(df[i].mean())

In [ ]:
# теперь разберемся с пропусками
df_nan = df.isnull().sum().reset_index()
df_nan.columns = ['col_name', 'quant']
column_nan = df_nan[df_nan.quant > 0].reset_index(drop = True)
column_nan.quant = column_nan.quant.apply(lambda x: round((x*100)/30471, 1))
print("Количество столбцов с пропущенными значениями: {}".format(column_nan.shape[0]))
print("Максимальный процент пропущенных данных по столбцам: {}%".format(column_nan.quant.max()))
print("Минимальный процент пропущенных данных по столбцам: {}%".format(column_nan.quant.min()))
print("Медианное значение пропущенных данных по столбцам: {}%".format(round(column_nan.quant.median(),1)))

### Данные почистили

In [ ]:
#блок слияния с макро 
zf = zipfile.ZipFile('../input/sberbank-russian-housing-market/macro.csv.zip') 
macro = pd.read_csv(zf.open('macro.csv'))
df.sort_index(inplace = True)
df.reset_index(inplace = True)
macro.timestamp = macro.timestamp.apply(lambda x: parse(x))   # спарсили дату
macro.reset_index(drop = True, inplace = True)    
df_macro = df.merge(macro, how = 'left', on = 'timestamp' )

In [ ]:
df_macro

In [ ]:
y = df['price_doc']                      #timestamp тоже удалить надо
x = df.drop(['price_doc'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [ ]:
%%time
cv = ShuffleSplit(n_splits=5,  random_state=42)

pipe = Pipeline([('regresion', RandomForestRegressor())])


param_grid =[
    {'regresion': [RandomForestRegressor(random_state = 42, n_jobs =-1)],
     'regresion__n_estimators': [int(x) for x in np.linspace(start = 25, stop = 30, num = 5)],
     'regresion__max_depth': [int(x) for x in np.linspace(start = 25, stop = 30, num = 5)] ,
    }
]

grid = GridSearchCV(pipe, param_grid, cv=cv, return_train_score=True)
grid.fit(X_train,y_train)

print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Средняя правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))
gridresults = pd.DataFrame(grid.cv_results_)
display(gridresults.sort_values(["rank_test_score"]).T)

In [ ]:
y=df.price_doc.tolist()

In [ ]:
df_corr=df.corr()

In [ ]:
df10=df_corr.nlargest(11, 'price_doc')

In [ ]:
df10['price_doc'].index

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
for i in df20['price_doc'].index.tolist():
    print(sum(df['%s'%i].isna()),i)

In [ ]:
df['num_room']

In [ ]:
df_rooms=df.corr().nlargest(21, 'num_room')

In [ ]:
df_rooms['num_room']

In [ ]:
from sklearn.linear_model import LinearRegression
df_replacena_train=df.loc[df['num_room'].notna()]

In [ ]:
df_replacena_train[['full_sq','num_room','life_sq']]

In [ ]:
df_linear=df_replacena_train[['full_sq','num_room']]

In [ ]:
df_linear_y=df_linear['num_room'].tolist()

In [ ]:
df_linear=df_linear.drop('num_room',axis=1)

In [ ]:
reg=LinearRegression()
reg.fit(df_linear,df_linear_y)

In [ ]:
df_na=df.loc[df['num_room'].isna()]

In [ ]:
df_linear_pred=df_na[['full_sq']]

In [ ]:
list_pred=reg.predict(df_linear_pred)
list_pred_rounded=[]
for i in list_pred:
    list_pred_rounded.append(round(i))


In [ ]:
df_na['num_room']=list_pred_rounded

In [ ]:
df_filled=pd.concat([df_na,df_replacena_train])

In [ ]:
df_filled

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,8))
sns.countplot(x="num_room", data=df_filled)
plt.ylabel('count')
plt.xlabel('num_room')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,8))
sns.countplot(x="timestamp", data=df_filled)
plt.ylabel('count')
plt.xlabel('timestamp')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df_filled['month'] = pd.DatetimeIndex(df_filled['timestamp']).month

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,8))
sns.countplot(x="month", data=df_filled)
plt.ylabel('count')
plt.xlabel('month')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
zf = zipfile.ZipFile('../input/sberbank-russian-housing-market/test.csv.zip') 
df_test = pd.read_csv(zf.open('test.csv'))

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,8))
sns.countplot(x="num_room", data=df_test)
plt.ylabel('count')
plt.xlabel('num_room')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df_test['month'] = pd.DatetimeIndex(df_test['timestamp']).month

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,8))
sns.countplot(x="month", data=df_test)
plt.ylabel('count')
plt.xlabel('month')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="month", data=df_filled)
plt.ylabel('count')
plt.xlabel('month')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df_filled['YearMonth'] = pd.DatetimeIndex(df_filled['timestamp']).map(lambda x: 100*x.year + x.month)

In [ ]:
df_test['YearMonth'] = pd.DatetimeIndex(df_test['timestamp']).map(lambda x: 100*x.year + x.month)

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="YearMonth", data=df_filled)
plt.ylabel('count')
plt.xlabel('month')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="YearMonth", data=df_test)
plt.ylabel('count')
plt.xlabel('month')
plt.show()